In [3]:
# !pip install scikit-learn

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Ruta del fitxer Excel
fitxer_excel = r'D:\34761933D\Downloads\Membres OGS.xlsx'

# 1. Carregar les dades
df = pd.read_excel(fitxer_excel)
# Seleccionar les columnes necessàries
df = df[['Denominació membre', 'Categorització partícip', 'Dret de vot']]
# Eliminar files amb valors nuls
df.dropna(inplace=True)

# 2. Preprocessament
# Combinar les columnes "Denominació membre" i "Membre partícip"
X = df['Denominació membre'] + ' ' + df['Dret de vot']
y = df['Categorització partícip']

# 3. Dividir les dades en conjunt d'entrenament i conjunt de prova
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Vectorització
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)  # Ajusta i transforma les dades d'entrenament
X_test_vect = vectorizer.transform(X_test)        # Només transforma les dades de prova

# 5. Entrenament del model
model = MultinomialNB()  # Model Naive Bayes
model.fit(X_train_vect, y_train)  # Entrena el model amb les dades vectoritzades

# 6. Predicció
y_pred = model.predict(X_test_vect)  # Realitza les prediccions

# 7. Avaluació
accuracy = accuracy_score(y_test, y_pred)  # Calcula la precisió del model
print(f'Precisió del model: {accuracy:.2f}')  # Mostra la precisió


Precisió del model: 0.82


In [65]:
# Noves dades per fer prediccions
noves_dades = [
    "El/la president/a del Consell Comarcal del Baix Llobregat",
    "Membre designat pel ministre competent"
]

# Vectoritzar les noves dades
noves_dades_vect = vectorizer.transform(noves_dades)

# Obtenir les prediccions
prediccions = model.predict(noves_dades_vect)

# Mostrar les prediccions
for denominacio, prediccio in zip(noves_dades, prediccions):
    print(f'Denominació: {denominacio} -> Predicció: {prediccio}')

Denominació: El/la president/a del Consell Comarcal del Baix Llobregat -> Predicció: SECTOR PÚBLIC LOCAL DE CATALUNYA
Denominació: Membre designat pel ministre competent -> Predicció: ADMINISTRACIÓ DE LA GENERALITAT DE CATALUNYA


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 16 stored elements and shape (1, 1588)>

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Ruta del fitxer Excel
fitxer_excel = r'D:\34761933D\Downloads\Membres OGS.xlsx'

# 1. Carregar les dades
df = pd.read_excel(fitxer_excel)
# Seleccionar les columnes necessàries
df = df[['Denominació membre', 'Categorització partícip', 'Denominació']]
# Eliminar files amb valors nuls
df.dropna(inplace=True)

# 2. Preprocessament
# Combinar les columnes "Denominació membre" i "Membre partícip"
X = df['Denominació membre'] + ' ' + str(df['Denominació'])
y = df['Categorització partícip']

# 3. Dividir les dades en conjunt d'entrenament i conjunt de prova
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# 4. Vectorització
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)  # Ajusta i transforma les dades d'entrenament
X_test_vect = vectorizer.transform(X_test)        # Només transforma les dades de prova

# 5. Entrenament del model amb Bosc Aleatori
model = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 arbres
model.fit(X_train_vect, y_train)  # Entrena el model

# 6. Predicció
y_pred = model.predict(X_test_vect)

# 7. Avaluació
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisió del model amb Random Forest: {accuracy:.2f}')


Precisió del model amb Random Forest: 0.91


In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
from sklearn.svm import SVC

# Ruta del fitxer Excel
fitxer_excel = r'D:\34761933D\Downloads\Membres OGS.xlsx'

# 1. Carregar les dades
df = pd.read_excel(fitxer_excel)

# Seleccionar les columnes necessàries
df = df[['Denominació membre', 'Categorització partícip', 'Denominació']]

# Eliminar files amb valors nuls
df.dropna(inplace=True)

# 2. Dividir les dades en conjunt d'entrenament i conjunt de prova
X_train, X_test, y_train, y_test = train_test_split(df[['Denominació membre', 'Denominació']], 
                                                    df['Categorització partícip'], test_size=0.2)

# 3. Vectoritzar "Denominació membre"
vectorizer_membre = CountVectorizer()
X_membre_train_vect = vectorizer_membre.fit_transform(X_train['Denominació membre'])
X_membre_test_vect = vectorizer_membre.transform(X_test['Denominació membre'])

# 4. Vectoritzar "Denominació"
vectorizer_denominacio = CountVectorizer()
X_denominacio_train_vect = vectorizer_denominacio.fit_transform(X_train['Denominació'])
X_denominacio_test_vect = vectorizer_denominacio.transform(X_test['Denominació'])

# 5. Concatenar les matrius vectoritzades d'entrenament i de prova
X_train_vect_combined = hstack([X_membre_train_vect, X_denominacio_train_vect])
X_test_vect_combined = hstack([X_membre_test_vect, X_denominacio_test_vect])

# 6. Entrenament del model amb SVM
model_svm = SVC(kernel='linear', random_state=42)
model_svm.fit(X_train_vect_combined, y_train)  # Entrena el model amb les dades combinades

# 7. Predicció
y_pred_svm = model_svm.predict(X_test_vect_combined)  # Utilitza les dades combinades de prova per fer prediccions

# 8. Avaluació
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'Precisió del model amb SVM: {accuracy_svm:.2f}')


Precisió del model amb SVM: 0.90


In [111]:
# Noves dades per fer prediccions (Denominació membre i Denominació)
noves_dades = pd.DataFrame({
    'Denominació membre': [
        "El/la president/a del Consell Comarcal del Baix Llobregat",
        "Membre designat per la junta d'Aelix, SL"
    ],
    'Denominació': [
        "Consorci Memorial dels Espais de la Batalla de l'Ebre",  # Exemple de denominació
        "Fundació Centre d'Informació i Documentació Internacionals a Barcelona (CIDOB)"  # Exemple de denominació
    ]
})

# Vectoritzar les noves dades
noves_dades_membre_vect = vectorizer_membre.transform(noves_dades['Denominació membre'])
noves_dades_denominacio_vect = vectorizer_denominacio.transform(noves_dades['Denominació'])

# Concatenar les matrius vectoritzades de les noves dades
noves_dades_vect_combined = hstack([noves_dades_membre_vect, noves_dades_denominacio_vect])

# Obtenir les prediccions utilitzant el model SVM entrenat
prediccions = model_svm.predict(noves_dades_vect_combined)

# Mostrar les prediccions
for (denominacio_membre, denominacio), prediccio in zip(noves_dades.values, prediccions):
    print(f'Denominació membre: {denominacio_membre}, Denominació: {denominacio} -> Predicció: {prediccio}')


Denominació membre: El/la president/a del Consell Comarcal del Baix Llobregat, Denominació: Consorci Memorial dels Espais de la Batalla de l'Ebre -> Predicció: ADMINISTRACIÓ DE LA GENERALITAT DE CATALUNYA
Denominació membre: Membre designat per la junta d'Aelix, SL, Denominació: Fundació Centre d'Informació i Documentació Internacionals a Barcelona (CIDOB) -> Predicció: SECTOR PRIVAT


In [117]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
from sklearn.ensemble import GradientBoostingClassifier

# Ruta del fitxer Excel
fitxer_excel = r'D:\34761933D\Downloads\Membres OGS.xlsx'

# 1. Carregar les dades
df = pd.read_excel(fitxer_excel)

# Seleccionar les columnes necessàries
df = df[['Denominació membre', 'Categorització partícip', 'Denominació']]

# Eliminar files amb valors nuls
df.dropna(inplace=True)

# 2. Dividir les dades en conjunt d'entrenament i conjunt de prova
X_train, X_test, y_train, y_test = train_test_split(df[['Denominació membre', 'Denominació']], 
                                                    df['Categorització partícip'], 
                                                    test_size=0.1, 
                                                    random_state=42)

# 3. Vectoritzar "Denominació membre"
vectorizer_membre = CountVectorizer()
X_membre_train_vect = vectorizer_membre.fit_transform(X_train['Denominació membre'])
X_membre_test_vect = vectorizer_membre.transform(X_test['Denominació membre'])

# 4. Vectoritzar "Denominació"
vectorizer_denominacio = CountVectorizer()
X_denominacio_train_vect = vectorizer_denominacio.fit_transform(X_train['Denominació'])
X_denominacio_test_vect = vectorizer_denominacio.transform(X_test['Denominació'])

# 5. Concatenar les matrius vectoritzades d'entrenament i de prova
X_train_vect_combined = hstack([X_membre_train_vect, X_denominacio_train_vect])
X_test_vect_combined = hstack([X_membre_test_vect, X_denominacio_test_vect])

# 6. Entrenament del model amb Gradient Boosting
model_gb = GradientBoostingClassifier(random_state=42)
model_gb.fit(X_train_vect_combined, y_train)  # Entrena el model amb les dades combinades

# 7. Predicció
y_pred_gb = model_gb.predict(X_test_vect_combined)  # Utilitza les dades combinades de prova

# 8. Avaluació
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f'Precisió del model amb Gradient Boosting: {accuracy_gb:.2f}')


Precisió del model amb Gradient Boosting: 0.92


In [123]:
import pandas as pd
from scipy.sparse import hstack

# Noves dades per fer prediccions (Denominació membre i Denominació)
noves_dades = pd.DataFrame({
    'Denominació membre': [
        "e)	Una persona representant de l’Institut d’Estudis Catalans",
        "Membre designat pel ministeri d'exteriors"
    ],
    'Denominació': [
        "Consorci Institut Ramon Llull",  # Exemple de denominació
        "Fundació Centre d'Informació i Documentació Internacionals a Barcelona (CIDOB)"  # Exemple de denominació
    ]
})

# Vectoritzar les noves dades
noves_dades_membre_vect = vectorizer_membre.transform(noves_dades['Denominació membre'])
noves_dades_denominacio_vect = vectorizer_denominacio.transform(noves_dades['Denominació'])

# Concatenar les matrius vectoritzades de les noves dades
noves_dades_vect_combined = hstack([noves_dades_membre_vect, noves_dades_denominacio_vect])

# Obtenir les prediccions utilitzant el model entrenat
prediccions = model_gb.predict(noves_dades_vect_combined)  # Assegura't de fer servir el model adequat

# Mostrar les prediccions
for (denominacio_membre, denominacio), prediccio in zip(noves_dades.values, prediccions):
    print(f'Denominació membre: {denominacio_membre}, Denominació: {denominacio} -> Predicció: {prediccio}')


Denominació membre: e)	Una persona representant de l’Institut d’Estudis Catalans, Denominació: Consorci Institut Ramon Llull -> Predicció: SECTOR PRIVAT
Denominació membre: Membre designat pel ministeri d'exteriors, Denominació: Fundació Centre d'Informació i Documentació Internacionals a Barcelona (CIDOB) -> Predicció: SECTOR PÚBLIC ESTATAL
